# Get Sentinel-3 SST
Explain how to obtain netCDF files usin EUMDAC.     
One needs to have EUMDAC credentials.

In [ ]:
import os
import json
import datetime
import shutil
import eumdac
import glob
import numpy as np
import matplotlib.pyplot as plt
import netCDF4

## Directories

In [5]:
# Create a download directory for our downloaded products
download_dir = "/data/Sentinel3/"
figdir = "/data/Sentinel3/figures"
os.makedirs(download_dir, exist_ok=True)

## Get access token

In [2]:
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

This token '0254b588-438d-34e4-b0eb-f421a5f9e344' expires 2022-12-23 17:21:02.824948


Show only the products corresponding to Sentinel-3 SST

In [6]:
for collectionID in datastore.collections:
    if ('Sentinel-3' in collectionID.title) & ('SST' in collectionID.title):
        print(f"{collectionID}: {collectionID.title}")

EO:EUM:DAT:0412: SLSTR Level 2 Sea Surface Temperature (SST) - Sentinel-3
EO:EUM:DAT:0582: SLSTR Level 2 Sea Surface Temperature (SST) (version BC003) - Sentinel-3 - Reprocessed


Select the collection of interest

In [7]:
collectionID = 'EO:EUM:DAT:0412'

# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(selected_collection.title)

SLSTR Level 2 Sea Surface Temperature (SST) - Sentinel-3


Get product based on the time period and the region of interest

In [8]:
start = datetime.datetime(2022, 12, 20)
end = datetime.datetime(2022, 12, 23)
domain = [-19., -13., 26., 30.]
roi = [[domain[0], domain[2]], [domain[1], domain[2]], [domain[1], domain[3]], 
       [domain[0], domain[3]], [domain[0], domain[2]]]

WKT = 'POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in roi]))

products = selected_collection.search(
    geo=WKT,
    dtstart=start, 
    dtend=end)

## Perform download

In [ ]:
for product in products:
    selected_product = datastore.get_product(product_id=str(product), collection_id=collectionID)

    with selected_product.open() as fsrc:
        outputfile = os.path.join(download_dir, fsrc.name)

        if os.path.isfile(outputfile):
            print(f"{outputfile} already downloaded")
        else:
            with open(outputfile, mode='wb') as fdst:
                print(f'Downloading {fsrc.name}')
                shutil.copyfileobj(fsrc, fdst)
                print(f'Download of product {fsrc.name} finished.')

Download of product S3B_SL_2_WST____20221222T231303_20221222T231603_20221223T014558_0180_074_130_0360_MAR_O_NR_003.SEN3.zip finished.
Download of product S3A_SL_2_WST____20221222T221110_20221222T221410_20221223T004131_0179_093_272_0360_MAR_O_NR_003.SEN3.zip finished.
Download of product S3A_SL_2_WST____20221222T110015_20221222T110315_20221222T131255_0179_093_265_2520_MAR_O_NR_003.SEN3.zip finished.
Download of product S3A_SL_2_WST____20221221T223721_20221221T224021_20221222T010810_0179_093_258_0360_MAR_O_NR_003.SEN3.zip finished.


Generate list of files

In [ ]:
datafilelist = glob.glob(os.path.join(download_dir, "*/*.nc"))
datafilelist

In [59]:
def read_sst(datafile):
    with netCDF4.Dataset(datafile) as nc:
        lon = nc.variables["lon"][:]
        lat = nc.variables["lat"][:]
        SST = nc.variables["sea_surface_temperature"][0,:,:] - 273.15
        qf = nc.variables["quality_level"][0,:,:]
        SSTq = np.ma.masked_where(qf != 5, SST)
    return lon, lat, SSTq

In [ ]:
for datafile in datafilelist:
    fname = os.path.basename(datafile).replace('.', '_').replace('_nc', '')
    lon, lat, SST = read_sst(datafile)
    
    fig = plt.figure(figsize=(12, 12))
    ax = plt.subplot(111)
    pcm = ax.pcolormesh(lon, lat, SST, cmap=plt.cm.RdYlBu_r, vmin=18, vmax=23.)
    ax.set_xlim(domain[0], domain[1])
    ax.set_ylim(domain[2], domain[3])
    plt.colorbar(pcm, extend="both")
    plt.savefig(os.path.join(figdir, fname), dpi=300, bbox_inches="tight")
    plt.close()
    fig.clf()

<ipython-input-65-a325d9ab86d2>:7: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  pcm = ax.pcolormesh(lon, lat, SST, cmap=plt.cm.RdYlBu_r)


'20221101233531-MAR-L2P_GHRSST-SSTskin-SLSTRB-20221102020644-v02_0-fv01_0'